In [1]:
# --- COMPLETE CONTEXT AND TEST SCRIPT (v2 - ALL FUNCTIONS INCLUDED) ---
# This script contains the final refactored code and a self-contained test case.
# Running this single cell will execute the test.

import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, date
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Markdown
import pprint
import io

# --- A. HELPER FUNCTIONS (Shared across tools) ---

def calculate_gain(price_series: pd.Series):
    if price_series.dropna().shape[0] < 2: return np.nan
    return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

def calculate_sharpe(return_series: pd.Series):
    if return_series.dropna().shape[0] < 2: return np.nan
    std_dev = return_series.std()
    if std_dev > 0 and std_dev != np.inf:
        return (return_series.mean() / std_dev) * np.sqrt(252)
    return np.nan

# --- B. THE CORE CALCULATION ENGINE (Headless, No UI) ---

def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
                          start_date, calc_period, fwd_period,
                          metric, rank_start, rank_end, benchmark_ticker):
    min_date_available = df_close_full.index.min()
    max_date_available = df_close_full.index.max()
    safe_start_date = max(start_date, min_date_available)
    safe_calc_end_date = min(start_date + calc_period, max_date_available)
    safe_viz_end_date = min(safe_calc_end_date + fwd_period, max_date_available)
    if safe_start_date >= safe_calc_end_date: return {'error': "Invalid date range."}
    calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
    calc_close = calc_close_raw.dropna(axis=1, how='all')
    if calc_close.shape[1] == 0 or len(calc_close) < 2: return {'error': "Not enough data in calc period."}

    metric_values = {}
    first_prices = calc_close.bfill().iloc[0]; last_prices = calc_close.ffill().iloc[-1]
    metric_values['Price'] = (last_prices / first_prices).dropna()
    daily_returns = calc_close.bfill().ffill().pct_change()
    mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
    metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
    valid_tickers = calc_close.columns
    calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    tr = np.maximum(calc_high - calc_low, abs(calc_high - df_close_full[valid_tickers].shift(1)), abs(calc_low - df_close_full[valid_tickers].shift(1)))
    atr = tr.ewm(alpha=1/14, adjust=False).mean()
    atrp = (atr / calc_close).mean()
    metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)
    
    sorted_tickers = metric_values[metric].sort_values(ascending=False)
    tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
    if not tickers_to_display: return {'error': "No tickers found for the selected rank."}
        
    normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
    normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
    actual_calc_end_ts = calc_close.index.max()
    portfolio_series = normalized_plot_data.mean(axis=1)
    portfolio_return_series = portfolio_series.pct_change()
    benchmark_price_series = df_close_full.get(benchmark_ticker)
    benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

    perf_data = {}
    perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
    perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
    perf_data['full_p_gain'] = calculate_gain(portfolio_series)
    perf_data['calc_p_sharpe'] = calculate_sharpe(portfolio_return_series.loc[:actual_calc_end_ts])
    perf_data['fwd_p_sharpe'] = calculate_sharpe(portfolio_return_series.loc[actual_calc_end_ts:])
    perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
    perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
    perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['calc_b_sharpe'] = calculate_sharpe(benchmark_return_series.loc[:actual_calc_end_ts])
    perf_data['fwd_b_sharpe'] = calculate_sharpe(benchmark_return_series.loc[actual_calc_end_ts:])
    perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)

    calc_end_prices = calc_close.ffill().iloc[-1]
    fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
    viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
    calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
    fwd_gains = (viz_end_prices / calc_end_prices) - 1
    results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
    if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
        benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
        results_df = pd.concat([results_df, benchmark_df_row])
    
    return { 'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data, 'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series, 'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts, 'safe_start_date': pd.to_datetime(df_close_full.loc[safe_start_date:safe_viz_end_date].index.min()), 'safe_viz_end_date': pd.to_datetime(df_close_full.loc[safe_start_date:safe_viz_end_date].index.max()), 'error': None }

# --- C. THE UI WRAPPER ---

def plot_walk_forward_analyzer(df_ohlcv, 
                               default_start_date=None, default_calc_period='3M', default_fwd_period='1M',
                               default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
                               default_benchmark_ticker='VOO'):
    print("Initializing Walk-Forward Analyzer...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
    df_ohlcv = df_ohlcv.sort_index()
    print("Pre-processing data (unstacking)...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
    start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
    calc_period_options = {'1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)}
    fwd_period_options = {'0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2), '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3)}
    calc_period_dropdown = widgets.Dropdown(options=calc_period_options.keys(), value=default_calc_period, description='Calc Period:')
    fwd_period_dropdown = widgets.Dropdown(options=fwd_period_options.keys(), value=default_fwd_period, description='Fwd Period:')
    metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
    rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
    rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
    benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    results_container = [None]
    
    fig = go.FigureWidget()
    max_traces = 50
    for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

    def update_plot(button_click):
        ticker_list_output.clear_output()
        start_date = pd.to_datetime(start_date_picker.value)
        calc_period = calc_period_options[calc_period_dropdown.value]; fwd_period = fwd_period_options[fwd_period_dropdown.value]
        metric = metric_dropdown.value; rank_start, rank_end = rank_start_dropdown.value, rank_end_dropdown.value
        benchmark_ticker = benchmark_ticker_input.value.strip().upper()
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return

        results = run_walk_forward_step(df_close_full, df_high_full, df_low_full, start_date, calc_period, fwd_period, metric, rank_start, rank_end, benchmark_ticker)
        
        if results['error']:
            with ticker_list_output: print(f"Error: {results['error']}")
            return
            
        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(results['tickers_to_display']):
                    ticker = results['tickers_to_display'][i]
                    trace.x, trace.y, trace.name = results['normalized_plot_data'].index, results['normalized_plot_data'][ticker], ticker
                    trace.visible, trace.showlegend = True, True
                else: trace.visible, trace.showlegend = False, False
            benchmark_trace = fig.data[max_traces]
            if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
                normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
                benchmark_trace.x, benchmark_trace.y = normalized_benchmark.index, normalized_benchmark
                benchmark_trace.name = f"Benchmark ({benchmark_ticker})"; benchmark_trace.visible = True
            else: benchmark_trace.visible = False
            portfolio_trace = fig.data[max_traces + 1]
            portfolio_trace.x, portfolio_trace.y = results['portfolio_series'].index, results['portfolio_series']
            portfolio_trace.name = 'Group Portfolio'; portfolio_trace.visible = True
            fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
            
        results_container[0] = results['results_df']
        
        with ticker_list_output:
            print(f"Analyzing from {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
            print(f"  - Ranking based on performance from {results['safe_start_date'].date()} to {results['actual_calc_end_ts'].date()}.")
            pprint.pprint(results['tickers_to_display'], width=120, compact=True)
            
            p = results['performance_data']
            rows = []
            rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
            if not np.isnan(p['full_b_gain']):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
                rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
            rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
            if not np.isnan(p['full_b_sharpe']):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
                rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})
            report_df = pd.DataFrame(rows).set_index('Metric')
            gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
            sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
            styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
            print("\n--- Strategy Performance Summary ---")
            display(styled_df)
            
    fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=700, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    controls_row1 = widgets.HBox([start_date_picker, calc_period_dropdown, fwd_period_dropdown])
    controls_row2 = widgets.HBox([metric_dropdown, rank_start_dropdown, rank_end_dropdown, benchmark_ticker_input, update_button])
    ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    update_plot(None)
    return results_container

# --- D. VERIFICATION TOOLS ---

def verify_group_tickers_walk_forward_calculation(df_ohlcv, tickers_to_verify, benchmark_ticker,
                                                  start_date, calc_period, fwd_period, export_csv=False):
    # This function is unchanged
    display(Markdown(f"## Verification Report for Portfolio vs. Benchmark"))
    display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`"))
    display(Markdown(f"**Benchmark Ticker:** `{benchmark_ticker}`"))
    period_options = { '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1), '0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2) }
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    start_date_ts = pd.to_datetime(start_date)
    calc_offset = period_options[calc_period]; fwd_offset = period_options[fwd_period]
    calc_end_date_ts_theoretical = start_date_ts + calc_offset
    fwd_end_date_ts_theoretical = calc_end_date_ts_theoretical + fwd_offset
    actual_calc_end_ts = df_close_full.loc[start_date_ts:calc_end_date_ts_theoretical].index.max()
    display(Markdown(f"**Analysis Start Date:** `{start_date_ts.date()}`"))
    display(Markdown(f"**Calculation Period End Date:** `{actual_calc_end_ts.date()}`"))
    display(Markdown(f"**Forward Period End Date:** `{fwd_end_date_ts_theoretical.date()}`"))
    portfolio_prices_raw_slice = df_close_full[tickers_to_verify].loc[start_date_ts:fwd_end_date_ts_theoretical]
    normalized_portfolio_prices = portfolio_prices_raw_slice.div(portfolio_prices_raw_slice.bfill().iloc[0])
    portfolio_value_series = normalized_portfolio_prices.mean(axis=1)
    try: benchmark_price_series = df_close_full[benchmark_ticker]
    except KeyError as e: print(f"---! ERROR: Ticker {e} not found !---"); return

    def print_verification_steps(title, price_series):
        display(Markdown(f"#### Verification for: `{title}`"))
        if price_series.dropna().shape[0] < 2: print("  - Not enough data points."); return {'gain': np.nan, 'sharpe': np.nan}
        start_price = price_series.bfill().iloc[0]; end_price = price_series.ffill().iloc[-1]
        gain = (end_price / start_price) - 1
        print(f"  - Start Value (on {price_series.first_valid_index().date()}): {start_price:,.4f}\n  - End Value   (on {price_series.last_valid_index().date()}): {end_price:,.4f}\n  - Gain = ({end_price:,.4f} / {start_price:,.4f}) - 1 = {gain:.2%}")
        returns = price_series.pct_change()
        mean_return = returns.mean(); std_return = returns.std()
        sharpe = (mean_return / std_return * np.sqrt(252)) if std_return > 0 and std_return != np.inf else np.nan
        print(f"\n  - Mean Daily Return: {mean_return:.6f}\n  - Std Dev of Daily Return: {std_return:.6f}\n  - Sharpe = ({mean_return:.6f} / {std_return:.6f}) * sqrt(252) = {sharpe:.2f}")
        return {'gain': gain, 'sharpe': sharpe}

    display(Markdown("### A. Calculation Period Analysis ('In-Sample')"))
    perf_calc_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[start_date_ts:actual_calc_end_ts])
    perf_calc_b = print_verification_steps(f"Benchmark ({benchmark_ticker})", benchmark_price_series.loc[start_date_ts:actual_calc_end_ts])
    display(Markdown("\n### B. Forward Period Analysis ('Moment of Truth')"))
    perf_fwd_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_calc_end_ts:fwd_end_date_ts_theoretical])
    perf_fwd_b = print_verification_steps(f"Benchmark ({benchmark_ticker})", benchmark_price_series.loc[actual_calc_end_ts:fwd_end_date_ts_theoretical])
    display(Markdown("\n### C. Full Period Analysis (Total)"))
    perf_full_p = print_verification_steps("Group Portfolio", portfolio_value_series)
    perf_full_b = print_verification_steps(f"Benchmark ({benchmark_ticker})", benchmark_price_series.loc[start_date_ts:fwd_end_date_ts_theoretical])
    display(Markdown("\n### D. Final Summary Table (matches analyzer output)"))
    rows = []
    rows.append({'Metric': 'Group Portfolio Gain', 'Full': perf_full_p['gain'], 'Calc': perf_calc_p['gain'], 'Fwd': perf_fwd_p['gain']})
    rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': perf_full_b['gain'], 'Calc': perf_calc_b['gain'], 'Fwd': perf_fwd_b['gain']})
    rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': perf_full_p['gain'] - perf_full_b['gain'], 'Calc': perf_calc_p['gain'] - perf_calc_b['gain'], 'Fwd': perf_fwd_p['gain'] - perf_fwd_b['gain']})
    rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': perf_full_p['sharpe'], 'Calc': perf_calc_p['sharpe'], 'Fwd': perf_fwd_p['sharpe']})
    rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': perf_full_b['sharpe'], 'Calc': perf_calc_b['sharpe'], 'Fwd': perf_fwd_b['sharpe']})
    rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': perf_full_p['sharpe'] - perf_full_b['sharpe'], 'Calc': perf_calc_p['sharpe'] - perf_calc_b['sharpe'], 'Fwd': perf_fwd_p['sharpe'] - perf_fwd_b['sharpe']})
    report_df = pd.DataFrame(rows).set_index('Metric')
    gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
    sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
    styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
    display(styled_df)
    if export_csv:
        export_df = pd.DataFrame({'Portfolio_Value_Normalized': portfolio_value_series, 'Portfolio_Return': portfolio_value_series.pct_change(), f'Benchmark_Price_{benchmark_ticker}': benchmark_price_series})
        filename = f"verification_group_tickers_{start_date_ts.strftime('%Y%m%d')}.csv"
        export_df.to_csv(filename, float_format='%.6f')
        print(f"\n✅ Detailed group verification data exported to '{filename}'")

def verify_ticker_ranking_metrics(df_ohlcv, 
                                  ticker, 
                                  start_date, 
                                  calc_period, 
                                  fwd_period, 
                                  export_csv=False):
    display(Markdown(f"## Verification Report for Ticker Ranking: `{ticker}`"))
    period_options = { '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1), '0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2) }
    try: df_ticker = df_ohlcv.loc[ticker].sort_index()
    except KeyError: print(f"---! ERROR: Ticker '{ticker}' not found !---"); return
    start_date_ts = pd.to_datetime(start_date)
    calc_offset = period_options[calc_period]; fwd_offset = period_options[fwd_period]
    calc_end_date_ts = start_date_ts + calc_offset; fwd_end_date_ts = calc_end_date_ts + fwd_offset
    display(Markdown(f"**Analysis Start Date:** `{start_date_ts.date()}`"))
    display(Markdown(f"**Requested Calculation Period:** `{start_date_ts.date()}` to `{calc_end_date_ts.date()}`"))
    display(Markdown(f"**Requested Forward Period:**   `{calc_end_date_ts.date()}` to `{fwd_end_date_ts.date()}`"))
    display(Markdown("### A. Calculation Period Analysis (for Ranking Metrics)"))
    calc_df = df_ticker.loc[start_date_ts:calc_end_date_ts].copy()
    if calc_df['Adj Close'].notna().sum() < 2: print("\n---! ERROR: Not enough data points !---"); return
    actual_calc_end_date = calc_df.index.max().date()
    display(Markdown(f"**Actual Dates Used:** `{calc_df.index.min().date()}` to `{actual_calc_end_date}`"))
    calc_gain = calculate_gain(calc_df['Adj Close'])
    calc_start_price = calc_df['Adj Close'].bfill().iloc[0]
    calc_end_price = calc_df['Adj Close'].ffill().iloc[-1]
    display(Markdown("#### `CalcGain` Verification:"))
    print(f"  - Calc Start Price: ${calc_start_price:.2f}\n  - Calc End Price:   ${calc_end_price:.2f}  <-- 'CalcPrice'\n  - CalcGain = {calc_gain:.2%}")
    display(Markdown("#### `MetricValue` Verification:"))
    price_metric = (calc_end_price / calc_start_price)
    print(f"\n1. Price Metric:\n   - Formula: Last Price / First Price = {price_metric:.4f}")
    daily_returns = calc_df['Adj Close'].bfill().ffill().pct_change()
    sharpe_ratio = calculate_sharpe(daily_returns)
    print(f"\n2. Sharpe Metric:\n   - Mean Daily Return: {daily_returns.mean():.6f}\n   - Std Dev Daily Return: {daily_returns.std():.6f}\n   - Annualized Sharpe = {sharpe_ratio:.4f}")
    print(f"\n3. Sharpe (ATR) Metric:")
    tr = np.maximum(calc_df['Adj High'] - calc_df['Adj Low'], abs(calc_df['Adj High'] - calc_df['Adj Close'].shift(1)), abs(calc_df['Adj Low'] - calc_df['Adj Close'].shift(1)))
    atr = tr.ewm(alpha=1/14, adjust=False).mean()
    atrp_series = atr / calc_df['Adj Close']
    atrp_mean = atrp_series.mean()
    sharpe_atr = (daily_returns.mean() / atrp_mean) if atrp_mean > 0 else 0
    print(f"   - Mean Daily Return: {daily_returns.mean():.6f} (same as above)\n   - Average ATR Percent (ATRP): {atrp_mean:.6f}\n   - Sharpe (ATR) = {sharpe_atr:.4f}")
    display(Markdown("\n### B. Forward Period Analysis (`FwdGain`)"))
    fwd_df = df_ticker.loc[actual_calc_end_date:fwd_end_date_ts].copy()
    fwd_gain = calculate_gain(fwd_df['Adj Close'])
    fwd_end_price = fwd_df['Adj Close'].ffill().iloc[-1] if fwd_gain is not np.nan else calc_end_price
    print(f"  - Fwd Start Price (Calc End Price): ${calc_end_price:.2f}\n  - Fwd End Price: ${fwd_end_price:.2f}\n  - FwdGain = {fwd_gain:.2%}")
    display(Markdown("\n### C. Final Summary Table"))
    summary_data = {'Metric': ['Price', 'Sharpe', 'Sharpe (ATR)'], 'Calculated Value': [f"{price_metric:.4f}", f"{sharpe_ratio:.4f}", f"{sharpe_atr:.4f}"], 'Corresponds To': ['`MetricValue`', '`MetricValue`', '`MetricValue`'], '---': ['---','---','---'], 'Gain Metric': ['Calc Period Gain', 'Forward Period Gain'], 'Gain Value': [f"{calc_gain:.2%}", f"{fwd_gain:.2%}"], 'Gain Corresponds To': ['`CalcGain`', '`FwdGain`']}
    summary_df = pd.DataFrame(summary_data)
    display(summary_df.style.hide(axis="index"))
    
    if export_csv:
        calc_df['Period'] = 'Calculation'; calc_df['Daily_Return'] = daily_returns; calc_df['True_Range'] = tr; calc_df['ATR_14'] = atr; calc_df['ATRP'] = atrp_series
        fwd_df['Period'] = 'Forward'
        combined_df = pd.concat([calc_df, fwd_df.iloc[1:]])
        filename = f"verification_ticker_{ticker}_{start_date_ts.strftime('%Y%m%d')}.csv"
        combined_df.to_csv(filename, float_format='%.6f')
        print(f"\n✅ Detailed ticker data exported to '{filename}'")

# --- 1. SAMPLE DATA (for reproducibility) ---
date_rng = pd.date_range(start='2023-01-01', end='2023-10-31', freq='B')
tickers = ['CVNA', 'VRT', 'APP', 'SMCI', 'IONQ', 'XPO', 'XP', 'AD', 'USM', 'MOD', 'VOO', 'QQQ']
data = []
np.random.seed(42) # for reproducibility
for ticker in tickers:
    price = 100 + (np.random.randn(len(date_rng)).cumsum() * (0.5 if ticker != 'SMCI' else 2.5))
    high = price + np.random.uniform(0, 2, size=len(date_rng))
    low = price - np.random.uniform(0, 2, size=len(date_rng))
    open_price = price + np.random.uniform(-1, 1, size=len(date_rng))
    volume = np.random.randint(100000, 5000000, size=len(date_rng))
    ticker_df = pd.DataFrame({'Date': date_rng,'Ticker': ticker,'Adj Open': open_price,'Adj High': high,'Adj Low': low,'Adj Close': price,'Volume': volume})
    data.append(ticker_df)
df_full = pd.concat(data)
df_OHLCV = df_full.set_index(['Ticker', 'Date'])

# --- 2. TEST EXECUTION ---
test_start_date = '2023-04-01'
test_calc_period = '6M'
test_fwd_period = '2W'
test_metric = 'Price'
test_rank_start = 1
test_rank_end = 10
test_benchmark = 'VOO'

print("--- RUNNING REFACTORED CODE TEST ---")
walk_forward_results = plot_walk_forward_analyzer(
    df_OHLCV,
    default_start_date=test_start_date,
    default_calc_period=test_calc_period,
    default_fwd_period=test_fwd_period,
    default_metric=test_metric,
    default_rank_start=test_rank_start,
    default_rank_end=test_rank_end,
    default_benchmark_ticker=test_benchmark
)
print("\n--- TEST COMPLETE ---")

--- RUNNING REFACTORED CODE TEST ---
Initializing Walk-Forward Analyzer...
Pre-processing data (unstacking)...


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'placeholder_0',
              'showlegend': False,
              'type': 'scatter',
              'uid': '22c20a95-637a-45e9-9c02-a6957b69a8fa',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_1',
              'showlegend': False,
              'type': 'scatter',
              'uid': '41a7319b-94a9-4510-88fc-e7089c3db81b',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_2',
              'showlegend': False,
              'type': 'scatter',
              'uid': '574a9ee2-a01d-4f53-82d5-334f02fc3ac5',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_3',
              'showlegend': False,
              'type': 


--- TEST COMPLETE ---


In [3]:
walk_forward_results_old = plot_walk_forward_analyzer_old(
    df_OHLCV,
    default_start_date=test_start_date,
    default_calc_period=test_calc_period,
    default_fwd_period=test_fwd_period,
    default_metric=test_metric,
    default_rank_start=test_rank_start,
    default_rank_end=test_rank_end,
    default_benchmark_ticker=test_benchmark
)

Initializing Walk-Forward Analyzer...
Pre-processing data (unstacking)...


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'placeholder_0',
              'showlegend': False,
              'type': 'scatter',
              'uid': 'fb6d3761-a624-4294-b79f-40eca21d2cb5',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_1',
              'showlegend': False,
              'type': 'scatter',
              'uid': '699ce74c-ff88-4f98-9016-a89551278fe1',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_2',
              'showlegend': False,
              'type': 'scatter',
              'uid': 'c434c13d-48bc-4122-ad90-132c0a449972',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_3',
              'showlegend': False,
              'type': 

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, date
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Markdown
import pprint

# --- Function 1: The Advanced Walk-Forward Analyzer (DEFINITIVE VERSION) ---
# --- Function 1: The Advanced Walk-Forward Analyzer (DEFINITIVE FINAL VERSION) ---
def plot_walk_forward_analyzer_old(df_ohlcv, 
                                   default_start_date=None,
                                   default_calc_period='3M', 
                                   default_fwd_period='1M',
                                   default_metric='Sharpe (ATR)',
                                   default_rank_start=1,
                                   default_rank_end=10,
                                   default_benchmark_ticker='VOO'):
    """
    Creates a robust, interactive walk-forward analysis plot.
    This version includes correct Wilder's ATR, correct equal-weighted Sharpe derived
    directly from the plotted portfolio line, and a correctly populated return DataFrame.
    """
    # --- All setup and widget code is unchanged... ---
    print("Initializing Walk-Forward Analyzer...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex):
        raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
    df_ohlcv = df_ohlcv.sort_index()
    print("Pre-processing data (unstacking)...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    df_close_full.index = pd.to_datetime(df_close_full.index)
    df_high_full.index = pd.to_datetime(df_high_full.index)
    df_low_full.index = pd.to_datetime(df_low_full.index)
    if isinstance(default_start_date, str):
        default_start_date = pd.to_datetime(default_start_date).date()
    min_date_available = df_close_full.index.min()
    max_date_available = df_close_full.index.max()
    if default_start_date is None:
        default_start_date = max_date_available.date() - pd.DateOffset(years=1)
    start_date_picker = widgets.DatePicker(description='Start Date:', value=default_start_date, disabled=False)
    calc_period_options = {'1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)}
    fwd_period_options = {'0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2), '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3)}
    calc_period_dropdown = widgets.Dropdown(options=calc_period_options.keys(), value=default_calc_period, description='Calc Period:')
    fwd_period_dropdown = widgets.Dropdown(options=fwd_period_options.keys(), value=default_fwd_period, description='Fwd Period:')
    metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
    rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
    rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
    benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    results_container = [None]
    fig = go.FigureWidget()
    max_traces = 50
    for i in range(max_traces):
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

    def calculate_gain(price_series: pd.Series):
        if price_series.dropna().shape[0] < 2: return np.nan
        return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

    def calculate_sharpe(return_series: pd.Series):
        if return_series.dropna().shape[0] < 2: return np.nan
        std_dev = return_series.std()
        if std_dev > 0:
            return (return_series.mean() / std_dev) * np.sqrt(252)
        return np.nan

    def update_plot(button_click):
        ticker_list_output.clear_output()
        start_date = pd.to_datetime(start_date_picker.value)
        calc_period = calc_period_options[calc_period_dropdown.value]; fwd_period = fwd_period_options[fwd_period_dropdown.value]
        metric = metric_dropdown.value; rank_start, rank_end = rank_start_dropdown.value, rank_end_dropdown.value
        benchmark_ticker = benchmark_ticker_input.value.strip().upper()
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return
        safe_start_date = max(start_date, min_date_available)
        safe_calc_end_date = min(start_date + calc_period, max_date_available)
        safe_viz_end_date = min(safe_calc_end_date + fwd_period, max_date_available)
        if safe_start_date >= safe_calc_end_date:
            with ticker_list_output: print(f"Error: Invalid date range."); return
        calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
        calc_close = calc_close_raw.dropna(axis=1, how='all')
        if calc_close.shape[1] == 0 or len(calc_close) < 2:
            with ticker_list_output: print("Error: Not enough data in calc period."); return

        metric_values = {}
        first_prices = calc_close.bfill().iloc[0]; last_prices = calc_close.ffill().iloc[-1]
        metric_values['Price'] = (last_prices / first_prices).dropna()
        daily_returns = calc_close.bfill().ffill().pct_change()
        mean_returns, std_returns = daily_returns.mean(), daily_returns.std()
        metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
        valid_tickers = calc_close.columns
        calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
        calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
        tr = np.maximum(calc_high - calc_low, abs(calc_high - df_close_full[valid_tickers].shift(1)), abs(calc_low - df_close_full[valid_tickers].shift(1)))
        atr = tr.ewm(alpha=1/14, adjust=False).mean()
        atrp = (atr / calc_close).mean()
        metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)
        
        sorted_tickers = metric_values[metric].sort_values(ascending=False)
        tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
        if not tickers_to_display:
            with ticker_list_output: print("Error: No tickers found."); return
        normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
        normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
        actual_calc_end_ts = calc_close.index.max()
        actual_viz_start_date = normalized_plot_data.index.min().date()
        actual_viz_end_date = normalized_plot_data.index.max().date()

        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(tickers_to_display):
                    ticker = tickers_to_display[i]
                    trace.x, trace.y, trace.name = normalized_plot_data.index, normalized_plot_data[ticker], ticker
                    trace.visible, trace.showlegend = True, True
                else: trace.visible, trace.showlegend = False, False
            benchmark_trace = fig.data[max_traces]
            if benchmark_ticker and benchmark_ticker in df_close_full.columns:
                benchmark_series = df_close_full[benchmark_ticker].loc[safe_start_date:safe_viz_end_date]
                if not benchmark_series.dropna().empty:
                    normalized_benchmark = benchmark_series / benchmark_series.bfill().iloc[0]
                    benchmark_trace.x, benchmark_trace.y = normalized_benchmark.index, normalized_benchmark
                    benchmark_trace.name = f"Benchmark ({benchmark_ticker})"; benchmark_trace.visible = True
                else: benchmark_trace.visible = False
            else: benchmark_trace.visible = False
            portfolio_trace = fig.data[max_traces + 1]
            portfolio_series = normalized_plot_data.mean(axis=1) # This is the data for the green line
            portfolio_trace.x, portfolio_trace.y = portfolio_series.index, portfolio_series
            portfolio_trace.name = 'Group Portfolio'; portfolio_trace.visible = True
            fig.layout.shapes = []
            fig.add_shape(type="line", x0=actual_calc_end_ts, y0=0, x1=actual_calc_end_ts, y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))

        # --- FIX 2: RESTORED THE RESULTS DATAFRAME BLOCK ---
        calc_start_prices = calc_close.bfill().iloc[0]; calc_end_prices = calc_close.ffill().iloc[-1]
        fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
        viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
        calc_gains = (calc_end_prices / calc_start_prices) - 1
        fwd_gains = (viz_end_prices / calc_end_prices) - 1
        results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'CalcPeriod': calc_period_dropdown.value, 'FwdPeriod': fwd_period_dropdown.value, 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
        if benchmark_trace.visible:
            bench_calc_prices = df_close_full[benchmark_ticker].loc[safe_start_date:actual_calc_end_ts]
            bench_fwd_prices = df_close_full[benchmark_ticker].loc[actual_calc_end_ts:safe_viz_end_date]
            bench_start_price = bench_calc_prices.bfill().iloc[0]
            bench_calc_end_price = bench_calc_prices.ffill().iloc[-1]
            bench_fwd_end_price = bench_fwd_prices.ffill().iloc[-1] if not bench_fwd_prices.empty and len(bench_fwd_prices) >= 2 else bench_calc_end_price
            bench_calc_gain = (bench_calc_end_price / bench_start_price) - 1
            bench_fwd_gain = (bench_fwd_end_price / bench_calc_end_price) - 1
            bench_metric_value = metric_values[metric].get(benchmark_ticker, np.nan)
            benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': bench_metric_value, 'CalcPrice': bench_calc_end_price, 'CalcGain': bench_calc_gain, 'CalcPeriod': calc_period_dropdown.value, 'FwdPeriod': fwd_period_dropdown.value, 'FwdGain': bench_fwd_gain}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
            results_df = pd.concat([results_df, benchmark_df_row])
        results_container[0] = results_df
        # --- END OF RESTORED BLOCK ---
        
        with ticker_list_output:
            print(f"Analyzing from {actual_viz_start_date} to {actual_viz_end_date}.")
            print(f"  - Ranking based on performance from {actual_viz_start_date} to {actual_calc_end_ts.date()}.")
            pprint.pprint(tickers_to_display, width=120, compact=True)
            
            # --- FIX 1: DERIVE SHARPE DIRECTLY FROM PLOTTED DATA ---
            # 'portfolio_series' is already defined above as `normalized_plot_data.mean(axis=1)`
            portfolio_return_series = portfolio_series.pct_change()
            
            benchmark_price_series = df_close_full.get(benchmark_ticker)
            benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

            calc_p_gain = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
            fwd_p_gain = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
            full_p_gain = calculate_gain(portfolio_series)
            
            calc_b_gain = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
            fwd_b_gain = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
            full_b_gain = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
            
            calc_p_sharpe = calculate_sharpe(portfolio_return_series.loc[:actual_calc_end_ts])
            fwd_p_sharpe = calculate_sharpe(portfolio_return_series.loc[actual_calc_end_ts:])
            full_p_sharpe = calculate_sharpe(portfolio_return_series)

            calc_b_sharpe = calculate_sharpe(benchmark_return_series.loc[:actual_calc_end_ts])
            fwd_b_sharpe = calculate_sharpe(benchmark_return_series.loc[actual_calc_end_ts:])
            full_b_sharpe = calculate_sharpe(benchmark_return_series)
            
            rows = []
            rows.append({'Metric': 'Group Portfolio Gain', 'Full': full_p_gain, 'Calc': calc_p_gain, 'Fwd': fwd_p_gain})
            if not np.isnan(full_b_gain):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': full_b_gain, 'Calc': calc_b_gain, 'Fwd': fwd_b_gain})
                rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': full_p_gain - full_b_gain, 'Calc': calc_p_gain - calc_b_gain, 'Fwd': fwd_p_gain - fwd_b_gain})
            rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': full_p_sharpe, 'Calc': calc_p_sharpe, 'Fwd': fwd_p_sharpe})
            if not np.isnan(full_b_sharpe):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': full_b_sharpe, 'Calc': calc_b_sharpe, 'Fwd': fwd_b_sharpe})
                rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': full_p_sharpe - full_b_sharpe, 'Calc': calc_p_sharpe - calc_b_sharpe, 'Fwd': fwd_p_sharpe - fwd_b_sharpe})
            report_df = pd.DataFrame(rows).set_index('Metric')
            gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
            sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
            styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
            print("\n--- Strategy Performance Summary ---")
            display(styled_df)

    fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=700, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    controls_row1 = widgets.HBox([start_date_picker, calc_period_dropdown, fwd_period_dropdown])
    controls_row2 = widgets.HBox([metric_dropdown, rank_start_dropdown, rank_end_dropdown, benchmark_ticker_input, update_button])
    ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    update_plot(None)
    return results_container

